In [2]:
import duckdb
import pandas as pd
from sodapy import Socrata
import config
import database_setup as dbs

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
with dbs.get_database_connection(config.DEV_DATABASE) as conn:
    stations_df = dbs.extract_dataset(config.MTA_CODE_STATIONS,
                                      limit=10000000)
    dbs.create_table(conn, stations_df, f"mta.stations")

In [9]:

client = Socrata("data.ny.gov",
                 config.NY_OPEN_DATA_API_TOKEN,
                 username=config.NY_OPEN_DATA_USERNAME,
                 password=config.NY_OPEN_DATA_PASSWORD)

results = client.get("39hk-dx4f", where="", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
%reload_ext sql
conn = dbs.get_database_connection(config.DEV_DATABASE)
%sql conn --alias duckdb


In [8]:
%%sql
select * from mta.stations

Running query in 'duckdb'

gtfs_stop_id,station_id,complex_id,division,line,stop_name,borough,cbd,daytime_routes,structure,gtfs_latitude,gtfs_longitude,north_direction_label,south_direction_label,ada,ada_northbound,ada_southbound,georeference,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc,:@computed_region_kjdx_g34t,ada_notes
R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,FALSE,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.912034, 40.775036]}",196,877,2137,None
R03,2,2,BMT,Astoria,Astoria Blvd,Q,FALSE,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,"{'type': 'Point', 'coordinates': [-73.917843, 40.770258]}",196,874,2137,None
R04,3,3,BMT,Astoria,30 Av,Q,FALSE,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.921479, 40.766779]}",196,874,2137,None
R05,4,4,BMT,Astoria,Broadway,Q,FALSE,N W,Elevated,40.76182,-73.925508,Astoria,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.925508, 40.76182]}",196,878,2137,None
R06,5,5,BMT,Astoria,36 Av,Q,FALSE,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.929575, 40.756804]}",196,878,2137,None
R08,6,6,BMT,Astoria,39 Av-Dutch Kills,Q,FALSE,N W,Elevated,40.752882,-73.932755,Astoria,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.932755, 40.752882]}",196,873,2137,None
R11,7,613,BMT,Astoria,Lexington Av/59 St,M,TRUE,N R W,Subway,40.76266,-73.967258,Queens,Downtown,0,0,0,"{'type': 'Point', 'coordinates': [-73.967258, 40.76266]}",749,749,2095,None
R13,8,8,BMT,Astoria,5 Av/59 St,M,TRUE,N R W,Subway,40.764811,-73.973347,Queens,Downtown,0,0,0,"{'type': 'Point', 'coordinates': [-73.973347, 40.764811]}",749,None,2095,None
R14,9,9,BMT,Broadway - Brighton,57 St-7 Av,M,TRUE,N Q R W,Subway,40.764664,-73.980658,Uptown,Downtown,1,1,1,"{'type': 'Point', 'coordinates': [-73.980658, 40.764664]}",749,742,2095,None
R15,10,10,BMT,Broadway - Brighton,49 St,M,TRUE,N R W,Subway,40.759901,-73.984139,Uptown,Downtown,2,1,0,"{'type': 'Point', 'coordinates': [-73.984139, 40.759901]}",749,742,2095,Uptown only


In [6]:
conn.sql("CREATE TABLE stations AS SELECT * FROM results_df")

CatalogException: Catalog Error: Table with name "stations" already exists!

In [3]:
with dbs.get_database_connection(config.DEV_DATABASE) as conn:
    conn.sql("DROP TABLE reduced_fare")
    conn.sql("DROP TABLE stations")
    conn.sql("DROP TABLE origin_destination_20230102")

In [1]:
conn.close()

NameError: name 'conn' is not defined

In [5]:
conn = duckdb.connect(config.DEV_DATABASE)

In [6]:
conn.sql("select * from stations")

┌──────────────┬────────────┬────────────┬──────────┬─────────────────────┬──────────────────────┬─────────┬─────────┬────────────────┬────────────┬───────────────┬────────────────┬───────────────────────┬───────────────────────┬─────────┬────────────────┬────────────────┬─────────────────────────────────────────────────────────┬─────────────────────────────┬─────────────────────────────┬─────────────────────────────┬─────────────┐
│ gtfs_stop_id │ station_id │ complex_id │ division │        line         │      stop_name       │ borough │   cbd   │ daytime_routes │ structure  │ gtfs_latitude │ gtfs_longitude │ north_direction_label │ south_direction_label │   ada   │ ada_northbound │ ada_southbound │                      georeference                       │ :@computed_region_yamh_8v7k │ :@computed_region_wbg7_3whc │ :@computed_region_kjdx_g34t │  ada_notes  │
│   varchar    │  varchar   │  varchar   │ varchar  │       varchar       │       varchar        │ varchar │ varchar │    varcha

In [7]:
conn.close()